<table style="background-color:#FFFFFF">   
  <tr>     
  <td><img src="https://upload.wikimedia.org/wikipedia/commons/9/95/Logo_EPFL_2019.svg" width="150x"/>
  </td>     
  <td>
  <h1> <b>CS-461: Foundation Models and Generative AI</b> </h1>
  Prof. Charlotte Bunne  
  </td>   
  </tr>
</table>

# 📚 Graded Assignment 1  
### CS-461: Foundation Models and Generative AI - Fall 2025  - Due: October 8, 23:59 CET

Welcome to the first graded assignment!
In this assignment, you will **implement and explore self-supervised learning** on a downsampled subset of the [ImageNet-1k dataset](https://www.image-net.org/), and evaluate how well your model generalizes **both in-distribution and out-of-distribution (OOD)**.  

---

## 🎯 Learning Objectives
By completing this assignment, you will learn to:
- Implement a custom **encoder** and **projection head** for images  
- Experiment with **data augmentations** for self-supervised learning  
- Train a model using a **self-supervised loss**  
- Evaluate learned representations with **k-NN** and **linear probes**  
- Assess **out-of-distribution (OOD) generalization** to unseen classes  
- Save, visualize, and submit results in a reproducible way  

---

## ⚡ Practical Notes
- **Dataset:**  
  - Training: 200 ImageNet classes, 500 images each (100k total)  
  - Validation: 200 ImageNet classes, 50 images each (10k total)  
  - **OOD dataset:** 200 unseen classes, 50 images each (10k total)  
- Use OOD only for **evaluation**, never for training.  
- Checkpoints and evaluation intervals are already set up — your main tasks are to fill in missing functions and customize the model.  
- Some helper utilities (e.g., dataset loaders, probes) are provided in `utils.py`.  

---

👉 **Deliverables:** You will submit:
- Your modified **`models.py`**  
- Trained weights in **`final_model.safetensors`**  
- A short **report.md** (max 500 words) — including **discussion of OOD results**  
- This completed notebook **CS461_Assignment1.ipynb**  

---

⚠️ **Important:** Don’t forget to fill in your **SCIPER number** and **full name** in Section 0, otherwise you will receive **0 points**.  

First, we import packages and set up the device. \
Feel free to add any additional packages you may need.

In [ ]:
# Automatically reloads modules when you make changes (useful during development)
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import shutil

import numpy as np
import math

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader
from safetensors.torch import save_model

import wandb
from torchsummary import summary

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 🆔 0. SCIPER Number and Name  

⚠️ **IMPORTANT!** ⚠️  
You **must** fill in your **SCIPER number** and **full name** below.  

This is **required for automatic grading**.  
If you do **not** provide this information, you will receive **0️⃣ (zero)** for this assignment. 

In [ ]:
SCIPER = 325403  # Replace with your SCIPER number
LAST_NAME = "Gotti"  # Replace with your last name
FIRST_NAME = "Bryan"  # Replace with your first name

## 1. Datasets & Utilities

- In the following, we will work with a subset of the ImageNet-1k dataset: color images downsampled to 64×64, covering 200 classes.
- The training set contains 500 images per class (100,000 images in total), and the validation set contains 50 images per class (10,000 images in total).
- The Out-Of-Distribution (OOD) datasets contain images from classes not present in the training set. It contains 50 images from 200 different classes (1,000 images in total).
- The purpose of these OOD datasets is to evaluate the generalization capabilities of the learned representations. You should not use it for training.
- During evalution, we will measure your model's performance on another OOD dataset (different from the one provided here), so make sure to not overfit on the provided OOD dataset.

<!-- Let's download/load it and define a default transformation turning a PIL Image into a `torch.tensor` -->
Make sure that you have access to the `/shared/CS461/cs461_assignment1_data/` folder. The folder structure should look like this:
```
cs461_assignment1_data/
└── train.npz
└── val.npz
└── ood.npz
```


Import dataset class and other utilities you developed in previous homeworks:

In [ ]:
from utils import ImageDatasetNPZ, default_transform, seed_all
from utils import run_knn_probe, run_linear_probe, extract_features_and_labels

For reproducibility, you can use the provided `seed_all` function to set the random seed for all relevant libraries (Python, NumPy, PyTorch).

In [ ]:
seed_all(42)  # For reproducibility, you can use any integer here

You probably want to implement custom data augmentations for the self-supervised learning method you choose. \
Feel free to swap the `default_transform` defined below and create multiple instances of datasets with different transforms.

In [ ]:
class SimCLRTransform:

    def __init__(self, size=32, s=0.5, blur_p=0.5):
        color_jitter = T.ColorJitter(0.8*s, 0.8*s, 0.8*s, 0.2*s)
        k = 3 if size <= 32 else 5
        base = [
            T.ToPILImage(),
            T.RandomResizedCrop(size=size, scale=(0.2, 1.0)),
            T.RandomHorizontalFlip(),
            T.RandomApply([color_jitter], p=0.8),
            T.RandomGrayscale(p=0.2),
            T.RandomApply([T.GaussianBlur(kernel_size=k, sigma=(0.1, 2.0))], p=blur_p),
            T.ToTensor()
        ]
        self.train_transform = T.Compose(base)

    def __call__(self, x):
        return self.train_transform(x), self.train_transform(x)

In [ ]:
data_dir = Path('./cs461_assignment1_data/')
checkpoints_dir = Path('checkpoints')
if not checkpoints_dir.exists():
    checkpoints_dir.mkdir(parents=True, exist_ok=False)
final_model_path = checkpoints_dir / 'model_final.safetensors'

In [ ]:
simclr_transform = SimCLRTransform(size=64)

train_datasets = [ImageDatasetNPZ(data_dir / 'train.npz', transform=simclr_transform) for i in range(4)]
train_dataset_no_augment = ImageDatasetNPZ(data_dir / 'train.npz', transform=default_transform)
val_dataset_no_augment = ImageDatasetNPZ(data_dir / 'val.npz', transform=default_transform)

You can split the provided OOD dataset into a training and validation set using the code below. \
You should not use the training split for actually training your models, but only for evaluation (e.g. kNN or linear probing).

In [ ]:
rng = np.random.RandomState(42)
ds_ood = ImageDatasetNPZ(data_dir / 'ood.npz', transform=default_transform)
ood_val_ratio = 0.2
train_mask = rng.permutation(len(ds_ood)) >= int(len(ds_ood) * ood_val_ratio)
ds_oods_train = torch.utils.data.Subset(ds_ood, np.where(train_mask)[0])
ds_oods_val = torch.utils.data.Subset(ds_ood, np.where(~train_mask)[0])

In [ ]:
batch_size = 256
num_workers = 4
pin_memory = True

def collate_fn(batch):
    xs1, xs2, ys = [], [], []
    for (x1, x2), y in batch:
        xs1.append(x1)
        xs2.append(x2)
        ys.append(y)
    return torch.stack(xs1), torch.stack(xs2), torch.tensor(ys)

In [ ]:
train_loaders = [DataLoader(td, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=True, collate_fn=collate_fn) for td in train_datasets]
train_loader_no_augment = DataLoader(train_dataset_no_augment, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=True)
val_loader_no_augment  = DataLoader(val_dataset_no_augment,  batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=False)

In [ ]:
oods_train_loader = DataLoader(ds_oods_train, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=True)
oods_val_loader = DataLoader(ds_oods_val, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=False)

# 2. Load Your Model

- Load your model from `models.py`.
- You will need to modify the `encoder` and `projection` modules, as the provided template implementation is only a placeholder.
- You SHOULD NOT change the `input_dim`, `input_channels`, and `feature_dim` parameters of the `ImageEncoder` class.
- You can use an existing architecture (e.g., ResNet, ViT) but you SHOULD NOT use any pre-trained weights.

In [ ]:
from models import ImageEncoder


model = ImageEncoder().to(device)
summary(model, input_size=(3, 64, 64))

## 3. Helpers for Training & Evaluation

We suggest you to implement the following helper functions to keep your training and evaluation loops clean and organized. \
- `training_step`: Performs a single training step (forward pass, loss computation, backward pass, optimizer step) and returns the loss value.
- `evaluation_step`: Evaluates the model on the validation dataset and returns the accuracy.

Depending on your specific requirements, you may also want to implement additional utility functions for tasks such as data loading, metric computation, and logging.

As you have seen from previous assignments, loss functions for self-supervised learning objectives can be quite complex. \
Feel free to implement any helper functions you may need to compute the loss.


In [ ]:
def training_step(*args, **kwargs):
    model, train_loaders, optimizer, custom_loss_function = args
    avg_loss = 0.
    for tl in train_loaders:
        for x1, x2, y in tqdm(tl):
            x1, x2 = x1.to(device), x2.to(device)

            with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                proj1 = model(x1)
                proj2 = model(x2)
            loss = custom_loss_function(proj1, proj2)
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
            avg_loss += loss.item()
    return { "loss": avg_loss / sum([len(tl) for tl in train_loaders]) }

In [ ]:
def evaluation_step(*args, **kwargs):
    model, train_loader_no_augment, val_loader_no_augment, oods_train_loader, oods_val_loader = args
    
    train_features, train_labels = extract_features_and_labels(model, train_loader_no_augment)
    test_features, test_labels = extract_features_and_labels(model, val_loader_no_augment)
    oods_train_features, oods_train_labels = extract_features_and_labels(model, oods_train_loader)
    oods_test_features, oods_test_labels = extract_features_and_labels(model, oods_val_loader)
    
    knn_accuracy = run_knn_probe(train_features, train_labels, test_features, test_labels)
    linear_accuracy = run_linear_probe(train_features, train_labels, test_features, test_labels)
    oods_knn_accuracy = run_knn_probe(oods_train_features, oods_train_labels, oods_test_features, oods_test_labels)
    oods_linear_accuracy = run_linear_probe(oods_train_features, oods_train_labels, oods_test_features, oods_test_labels)

    return { "knn_accuracy": knn_accuracy, "linear_accuracy": linear_accuracy, "oods_knn_accuracy": oods_knn_accuracy, "oods_linear_accuracy": oods_linear_accuracy }

In [ ]:
def custom_loss_function(z1, z2, tau=0.5):
    """
    Computes NT-Xent loss.
    z1: (batch_size, feature_dim) tensor of normalized projection vectors
    z2: (batch_size, feature_dim) tensor of normalized projection vectors
    returns: loss (scalar)
    """
    B, d = z1.shape
    z = torch.cat([z1, z2], dim=0)              # (2B, d)
    sim = (z @ z.t()) / tau                     # (2B, 2B)
    mask = torch.eye(2*B, dtype=torch.bool, device=z.device)
    sim.masked_fill_(mask, -1e9)
    targets = torch.arange(B, device=z.device)
    targets = torch.cat([targets + B, targets], dim=0)
    return F.cross_entropy(sim, targets)

# 4. Optimizer Configuration

In [ ]:
total_epochs = 50  # Adjust the number of epochs as needed
warmup_epochs = 10

def lr_lambda(epoch):
    if epoch < warmup_epochs:
        return (epoch + 1) / float(warmup_epochs)
    t = (epoch - warmup_epochs) / float(total_epochs - warmup_epochs)
    return 0.0 + 0.5 * (1 - 0.0) * (1 + math.cos(math.pi * t))

optimizer = torch.optim.AdamW(model.parameters(), lr=0.6, weight_decay=0.0)
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# Feel free to adapt and add more arguments
# lr = 1e-3
# weight_decay = 5e-2
# lr_step_size = 10
# lr_gamma = 0.1
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=lr_step_size, gamma=lr_gamma)

# 5. Training Loop

Adapt your training configuration and implement the training loop. \
You probably want to save model checkpoints and evaluate the model on the validation set at regular intervals.

In [ ]:
n_epochs = total_epochs
eval_interval = 3  # Evaluate the model every 'eval_interval' epochs
save_interval = 10  # Save the model every 'save_interval' epochs

In [ ]:
with wandb.init(project="cs461_assignment1") as run:

    all_train_stats = []
    all_val_stats = []

    for epoch in range(n_epochs):
        # TODO: Implement the training and evaluation loop
        model.train()
        train_stats = training_step(model, train_loaders, optimizer, custom_loss_function)
        all_train_stats.append(train_stats)

        log_dict = {**train_stats, 'epoch': epoch + 1}

        lr_scheduler.step()

        if (epoch + 1) % eval_interval == 0:
            model.eval()
            val_stats = evaluation_step(model, train_loader_no_augment, val_loader_no_augment, oods_train_loader, oods_val_loader)
            all_val_stats.append(val_stats)
            log_dict.update(val_stats)
            print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {train_stats['loss']:.4f}, Val kNN-5 Accuracy: {val_stats['knn_accuracy']:.2f}, Val Linear Accuracy: {val_stats['linear_accuracy']:.2f}, OOD kNN-5 Accuracy: {val_stats['oods_knn_accuracy']:.2f}, OOD Linear Accuracy: {val_stats['oods_linear_accuracy']:.2f}")

        if (epoch + 1) % save_interval == 0:
            checkpoint_path = checkpoints_dir / f'model_epoch_{epoch+1}.safetensors'
            save_model(model, checkpoint_path)
            print(f"Model checkpoint saved at {checkpoint_path}")

        wandb.log(log_dict)


    # Save the final model
    save_model(model, final_model_path)

# 6. Visualize Results

To better understand the performance of your trained model, visualize some results. \
You can visualize:
- Sample images from the validation set along with their predicted labels.
- Training and validation loss curves over epochs.

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(12, 5))
axes[0].plot(range(1, len(all_train_stats)+1), [s["loss"] for s in all_train_stats])
axes[0].set_xlabel('Epoch (1 unit corresponds to 1 epoch)')
axes[0].set_ylabel('Average NT-Xent Loss')

axes[1].plot(range(1, len(all_val_stats)+1), [s["knn_accuracy"] for s in all_val_stats])
axes[1].set_xlabel(f'Epoch (1 unit corresponds to {eval_interval} epochs)')
axes[1].set_ylabel('k-NN Accuracy')
axes[1].set_ylim(0, 1)

axes[2].plot(range(1, len(all_val_stats)+1), [s["linear_accuracy"] for s in all_val_stats])
axes[2].set_xlabel(f'Epoch (1 unit corresponds to {eval_interval} epochs)')
axes[2].set_ylabel('Linear Accuracy')
axes[2].set_ylim(0, 1)

axes[3].plot(range(1, len(all_val_stats)+1), [s["oods_knn_accuracy"] for s in all_val_stats])
axes[3].set_xlabel(f'Epoch (1 unit corresponds to {eval_interval} epochs)')
axes[3].set_ylabel('OOD k-NN Accuracy')
axes[3].set_ylim(0, 1)

axes[4].plot(range(1, len(all_val_stats)+1), [s["oods_linear_accuracy"] for s in all_val_stats])
axes[4].set_xlabel(f'Epoch (1 unit corresponds to {eval_interval} epochs)')
axes[4].set_ylabel('OOD Linear Accuracy')
axes[4].set_ylim(0, 1)

plt.show()

# 7. Submission Instructions

You must submit the following files:
- `models.py`: Contains the implementation of your model architecture.
- `final_model.safetensors`: The trained model weights saved in the safetensors format.
- `report.md`: A brief report summarizing your approach, design choices, and results.
- `CS461_Assignment1.ipynb`: The Jupyter notebook containing your code and explanations. Make sure to save your progress before running the cell below.

You will submit your assignment under a single folder named `/home/cs461_assignment1_submission` containing the above files. \
Make sure to replace `<SCIPER>`, `<LAST_NAME>`, and `<FIRST_NAME>` with your actual SCIPER number, last name, and first name respectively. \
The following cell will help you move the files into the submission folder.

In [ ]:
work_dir = Path('.')
output_dir = Path.home() / 'cs461_assignment1_submission'

if not output_dir.exists():
    output_dir.mkdir(parents=True, exist_ok=False)
    
shutil.copy(final_model_path, output_dir / 'final_model.safetensors')
shutil.copy(work_dir / 'models.py', output_dir / 'models.py')
shutil.copy(work_dir / 'CS461_Assignment1.ipynb', output_dir / 'CS461_Assignment1.ipynb')
shutil.copy(work_dir / 'report.md', output_dir / 'report.md')

Check that all required files are present in the submission folder before running the cell below.

In [ ]:
assert SCIPER is not None and LAST_NAME is not None and FIRST_NAME is not None, "Please set your SCIPER, LAST_NAME, and FIRST_NAME variables."

list_of_files = ['final_model.safetensors', 'models.py', 'CS461_Assignment1.ipynb', 'report.md']
files_found = all((output_dir / f).exists() for f in list_of_files)
assert files_found, f"One or more required files are missing in the submission folder: {list_of_files}"


You can test whether your submission folder is appropriately structured by using the `eval.py`:
```bash
python eval.py
```

In [ ]:
### Uncomment the line below to run the evaluation script and check your model's performance

!python eval.py

---
🎉 **Congratulations!**  
You’ve completed Assignment 1. Good luck, and don’t forget to double-check your submission!